In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
gap_train= pd.read_pickle('./temp_result/train_kaggle_processed_PPA_PCA_PPA')
gap_test= pd.read_pickle('./temp_result/test_kaggle_processed_PPA_PCA_PPA')
gap_valid= pd.read_pickle('./temp_result/valid_kaggle_processed_PPA_PCA_PPA')

In [3]:
def label(A,B):
    if A is True:
        return 0
    if B is True:
        return 1
    return 2

In [4]:
import torch
def compute_loss(sub_df,test_data):
    pred = torch.Tensor(np.log(sub_df.loc[:,['A','B','NEITHER']].values))
    label = torch.LongTensor(list(test_data.label))
    loss = torch.nn.NLLLoss()
    loss_value = loss(pred,label).item()
    return loss_value

In [5]:
train_data = gap_train.drop(columns = ['ID', 'Text', 'Pronoun', 'vector','Pronoun-offset', 'A', 'A-offset',
       'B', 'B-offset', 'URL', 'tokens', 'token_map',
       'sentence_map','A_idx', 'B_idx', 'pron_idx'])
train_data.A_vector = train_data.A_vector.map(lambda x:np.mean(x,axis = 0))
train_data.B_vector = train_data.B_vector.map(lambda x:np.mean(x,axis = 0))
train_data.pron_vector = train_data.pron_vector.map(lambda x:np.mean(x,axis = 0))
train_data["product_vector_A"] = train_data.A_vector*train_data.pron_vector
train_data["product_vector_B"] = train_data.B_vector*train_data.pron_vector
train_data["label"] = train_data.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
train_data = train_data.drop(columns= ["A-coref","B-coref"])

In [6]:
test_data = gap_test.drop(columns = ['ID', 'Text', 'Pronoun', 'vector','Pronoun-offset', 'A', 'A-offset',
       'B', 'B-offset', 'URL', 'tokens', 'token_map',
       'sentence_map','A_idx', 'B_idx', 'pron_idx'])
test_data.A_vector = test_data.A_vector.map(lambda x:np.mean(x,axis = 0))
test_data.B_vector = test_data.B_vector.map(lambda x:np.mean(x,axis = 0))
test_data.pron_vector = test_data.pron_vector.map(lambda x:np.mean(x,axis = 0))
test_data["product_vector_A"] = test_data.A_vector*test_data.pron_vector
test_data["product_vector_B"] = test_data.B_vector*test_data.pron_vector
test_data["label"] = test_data.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
test_data = test_data.drop(columns= ["A-coref","B-coref"])

In [7]:
valid_data = gap_valid.drop(columns = ['ID', 'Text', 'Pronoun', 'vector','Pronoun-offset', 'A', 'A-offset',
       'B', 'B-offset', 'URL', 'tokens', 'token_map',
       'sentence_map','A_idx', 'B_idx', 'pron_idx'])
valid_data.A_vector = valid_data.A_vector.map(lambda x:np.mean(x,axis = 0))
valid_data.B_vector = valid_data.B_vector.map(lambda x:np.mean(x,axis = 0))
valid_data.pron_vector = valid_data.pron_vector.map(lambda x:np.mean(x,axis = 0))
valid_data["product_vector_A"] = valid_data.A_vector*valid_data.pron_vector
valid_data["product_vector_B"] = valid_data.B_vector*valid_data.pron_vector
valid_data["label"] = valid_data.apply(lambda x:label(x["A-coref"],x["B-coref"]),axis = 1)
valid_data = valid_data.drop(columns= ["A-coref","B-coref"])

In [8]:
columns = train_data.columns[:-1]
X_train = np.concatenate([np.array(list(train_data[col])).reshape(train_data.shape[0],-1) for col in columns],axis = 1)
y_train = list(train_data.label)   

In [9]:
X_valid = np.concatenate([np.array(list(valid_data[col])).reshape(valid_data.shape[0],-1) for col in columns],axis = 1)
y_valid = list(valid_data.label)   

In [10]:
X_test = np.concatenate([np.array(list(test_data[col])).reshape(test_data.shape[0],-1) for col in columns],axis = 1)
y_test= list(test_data.label)   

In [11]:
train_data.head()

,A_dist,B_dist,A_pos,B_pos,pron_pos,A_vector,B_vector,pron_vector,product_vector_A,product_vector_B,label
0,0.014,0.008,0.444444,0.464052,0.490196,"[-0.16186124, 0.10288441, -0.13267949, 0.09381...","[0.46889028, 0.719243, -0.12934478, 0.2970564,...","[0.6202348, -0.49657542, 0.5204739, -0.7274118...","[-0.10039197, -0.051089868, -0.06905621, -0.06...","[0.29082206, -0.3571584, -0.06732058, -0.21608...",1
1,0.026,0.014,0.253623,0.275362,0.300725,"[-0.1530692, -0.45684996, -0.10911498, -0.1501...","[0.38673186, 0.017928414, 0.034514368, 0.03494...","[0.6709301, -0.5158502, 0.73226666, -0.5642047...","[-0.10269873, 0.23566614, -0.07990126, 0.08469...","[0.25947005, -0.009248376, 0.025273722, -0.019...",0
2,0.024,0.008,0.408333,0.475000,0.508333,"[0.15905416, 0.3745935, 0.068947755, 0.3243495...","[0.14589697, 0.29442316, -0.3740158, 0.4216888...","[0.37514523, -0.4730158, 0.10452151, -0.033686...","[0.05966841, -0.17718863, 0.007206524, -0.0109...","[0.054732554, -0.1392668, -0.039092697, -0.014...",0
3,0.056,-0.004,0.222951,0.321311,0.314754,"[0.05937913, 0.2836007, -0.36608246, 0.7199421...","[-0.6133348, -0.49770007, -0.5726678, 0.862215...","[0.6802153, -0.041674256, 0.36125743, 0.468463...","[0.040390592, -0.011818848, -0.13225001, 0.337...","[-0.4171997, 0.02074128, -0.2068805, 0.4039163...",1
4,-0.006,-0.056,0.473118,0.607527,0.456989,"[0.19817783, 0.30553705, 0.70740294, -1.038957...","[-0.02850709, -0.038969748, 0.3848874, -0.7527...","[0.31947732, -0.5680447, 1.1254325, -0.7034413...","[0.06331332, -0.17355871, 0.79613423, 0.730845...","[-0.009107368, 0.02213656, 0.43316478, 0.52953...",0


In [12]:
'''
import pickle
f = open( "./temp_result/base_model_data_PPA", "wb" )
pickle.dump(X_train,  f)
pickle.dump(y_train,  f)
pickle.dump(X_valid,  f)
pickle.dump(y_valid,  f)
pickle.dump(X_test,  f)
f.close()
'''

In [13]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(X_train, y_train)


pred_lr = lr.predict_proba(X_test)

sub_df = pd.read_csv("./test_and_submit/sample_submission_stage_1.csv")
sub_df.loc[:, ['A','B','NEITHER']] = pred_lr

sub_df.head()


sub_df.to_csv("./test_and_submit/submission+model+lr@"+str(datetime.datetime.now())+".csv", index=False)

/home/bao/anaconda3/envs/EPFL/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [14]:
compute_loss(sub_df,test_data)

0.9079073667526245

In [15]:
from sklearn.svm import SVC
svm = SVC(C = 7.0,verbose=True,probability = True,gamma = "auto").fit(X_train, y_train)


pred_svm = svm.predict_proba(X_test)

sub_df = pd.read_csv("./test_and_submit/sample_submission_stage_1.csv")
sub_df.loc[:, ['A','B','NEITHER']] = pred_svm


[LibSVM]

In [16]:
sub_df.to_csv("./test_and_submit/submission+model+svm@"+str(datetime.datetime.now())+".csv", index=False)
sub_df.head()

,ID,A,B,NEITHER
0,development-1,0.750300,0.175348,0.074353
1,development-2,0.785681,0.117283,0.097036
2,development-3,0.046601,0.928087,0.025312
3,development-4,0.035881,0.763022,0.201097
4,development-5,0.027071,0.786711,0.186218


In [17]:
compute_loss(sub_df,test_data)

0.5227930545806885